In [1]:
import pandas as pd

In [2]:
data_folder = 'data/sample_data'

B1 = pd.read_csv(data_folder + '/B1.csv', header=None)
print(B1.shape)
B1

(5, 7)


,0,1,2,3,4,5,6
0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,-1.0,-1.0,-1.0,0.0,0.0
2,0.0,1.0,1.0,0.0,0.0,-1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,-1.0
4,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [3]:
B2 = pd.read_csv(data_folder + '/B2.csv', header=None)
print(B2.shape)
B2

(2, 7)


,0,1,2,3,4,5,6
0,0.0,0.0,1.0,-1.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,-1.0,0.0,1.0


In [6]:
from sclibrary.read_incidence_matrix import ReadIncidenceMatrix
from sclibrary.simplicial_complex import SimplicialComplexNetwork

data = ReadIncidenceMatrix(B1, B2)
data.get_data_summary()
print("Nodes", data.get_nodes())
print("Edges", data.get_edge_list(rank=1))
print("Triangle Edges", data.get_edge_list(rank=2))
print("Adjacency", data.get_adjacency_matrix())

edges = data.get_edge_list()
print(edges)
sc = SimplicialComplexNetwork(edges, pos=None)

number of nodes:  5
number of edges:  7
number of triangles:  2
Nodes [0, 1, 2, 3, 4]
Edges [(0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (2, 3), (3, 4)]
Triangle Edges [[1, 2, 3], [1, 3, 4]]
Adjacency [[ 0 -1 -1  0  0]
 [ 1  0 -1 -1 -1]
 [ 1  1  0 -1  0]
 [ 0  1  1  0 -1]
 [ 0  1  0  1  0]]
[(0, 1), (0, 2), (1, 2), (1, 3), (1, 4), (2, 3), (3, 4), [1, 2, 3], [1, 3, 4]]


In [144]:
L1 = sc.hodge_laplacian_matrix(rank=1)
L1L = sc.lower_laplacian_matrix(rank=1)
L1U = sc.upper_laplacian_matrix(rank=1)
L1U + L1L

array([[ 2.,  1., -1., -1., -1.,  0.,  0.],
       [ 1.,  2.,  1.,  0.,  0., -1.,  0.],
       [-1.,  1.,  3.,  0.,  1.,  0.,  0.],
       [-1.,  0.,  0.,  4.,  0.,  0.,  0.],
       [-1.,  0.,  1.,  0.,  3.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  3., -1.],
       [ 0.,  0.,  0.,  0.,  0., -1.,  3.]], dtype=float32)

In [182]:
import numpy as np

def get_harmonic_eigenvectors(hodgle_lap_mat: np.ndarray) -> tuple:
    eigenvectors, eigenvalues = _get_eigendecomposition(hodgle_lap_mat)
    # get columns with zero eigenvalues
    u_h = eigenvectors[:, np.where(eigenvalues == 0)[0]]
    return u_h, eigenvalues

def get_curl_eigenvectors(upper_lap_mat: np.ndarray) -> tuple:
    eigenvectors, eigenvalues = _get_eigendecomposition(upper_lap_mat)
    # get columns with non-zero eigenvalues
    u_c = eigenvectors[:, np.where(eigenvalues != 0)[0]]
    return u_c, eigenvalues


def get_gradient_eigenvectors(lower_lap_mat: np.ndarray) -> tuple:
    eigenvectors, eigenvalues = _get_eigendecomposition(lower_lap_mat)
    # get columns with non-zero eigenvalues
    u_g = eigenvectors[:, np.where(eigenvalues != 0)[0]]
    return u_g, eigenvalues

def _get_eigendecomposition(lap_mat: np.ndarray, tolerance=1e-03) -> tuple:
    eigenvalues, eigenvectors = np.linalg.eig(lap_mat)
    eigenvalues[eigenvalues < tolerance] = 0
    lambda_matrix = np.diag(eigenvalues)
    # verify L(k) = U(k) * lambda(k) * U(k).T
    assert np.allclose(
        np.rint(eigenvectors @ lambda_matrix @ eigenvectors.T),
        lap_mat,
    )
    return eigenvectors, eigenvalues

In [183]:
u_h, lambda_mat_h = get_harmonic_eigenvectors(L1)
u_c, lambda_mat_c = get_curl_eigenvectors(L1U)
u_g, lambda_mat_g = get_gradient_eigenvectors(L1L)

(7, 7)

In [137]:
# flow geenration
flow = np.random.rand(L1.shape[0], 1)
print("flow", flow)
# analyze frequency components
flow_h = u_h.T @ flow
flow_g = u_g.T @ flow
flow_c = u_c.T @ flow
print("flow_h", flow_h)
print("flow_g", flow_g)
print("flow_c", flow_c)

flow [[0.31004]
 [0.03894]
 [0.10792]
 [0.1813 ]
 [0.45826]
 [0.18989]
 [0.53967]]
flow_h [[ 0.18668]
 [-0.67182]
 [-0.24841]
 [ 0.21872]
 [-0.21247]
 [-0.18961]
 [ 0.05168]]
flow_g [[ 0.18668]
 [-0.67182]
 [-0.24841]
 [ 0.21872]
 [ 0.21247]
 [ 0.16781]
 [-0.18515]]
flow_c [[ 0.1434 ]
 [-0.18961]
 [-0.05168]
 [-0.40154]
 [ 0.37462]
 [ 0.31004]
 [ 0.03894]]
